In [ ]:
# установка внешних программ и библиотек на виртуальную машину
!git clone --recurse-submodules https://github.com/KarlsruheMIS/KaMIS.git
%cd KaMIS
!./compile_withcmake.sh

%cd ..


!pip3 install python-sat
#!sudo apt install minisat
!git clone https://github.com/arminbiere/kissat.git
%cd kissat
!./configure
!make

%cd ..

!cp KaMIS/deploy/redumis redumis
!cp KaMIS/deploy/online_mis online_mis
!cp kissat/build/kissat kissat1



Cloning into 'KaMIS'...
remote: Enumerating objects: 1449, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 1449 (delta 27), reused 21 (delta 19), pack-reused 1399
Receiving objects: 100% (1449/1449), 12.76 MiB | 10.95 MiB/s, done.
Resolving deltas: 100% (818/818), done.
/content/KaMIS
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_FUNROLL_LOOPS
--

In [1]:
# функции из библиотеки
# раскраска графа в формате  DIMACS, преобразование в CNF

import math
import numpy as np
from numpy import linalg as LA
import networkx as nx

#import matplotlib.pylab as plt

#from google.colab import files 
#files.upload() 
#files.download("file.txt")

#https://pysathq.github.io/
#!pip install python-sat
#import pysat
from pysat.formula import CNF
from pysat.solvers import *

import os
import sys
import random
from zipfile import ZipFile
#from tqdm import tqdm_notebook as tqdm
from itertools import combinations, permutations
import glob

out_dir = ""

class Utils:
    
    def read_dimacs_graph(file = 'graph.col'):
        
        if not (os.path.exists(file) and os.path.getsize(file) > 0):        
            raise Exception("File " + file + " not found")
        
        nodes = []    
        edges = []
        labels = []
        got_labels = False
        nnodes = nedges = 0
        
        with open(file, 'r') as f:
            for line in f:
                line = [l.strip() for l in line.split(' ')]
                if line[0] == 'c': #comment
                    continue
                elif line[0] == 'p': #problem
                    nnodes = int(line[2])
                    nedges = int(line[3])
                    nodes = list(range(1, nnodes + 1))
                    labels = [0] * nnodes
                elif line[0] == 'e': #edge
                    edges.append((int(line[1]), int(line[2])))
                elif line[0] == 'l':
                    labels[int(line[1]) - 1] = int(line[2])
                    got_labels = True

        if got_labels:        
            nodes = [(n, {'c' : l}) for n, l in zip(nodes, labels)]

        g = nx.Graph()
        g.add_nodes_from(nodes)
        g.add_edges_from(edges)
        return g

    def write_dimacs_graph(file = 'graph.col', g = nx.Graph(), comments = []):
        with open(file, 'w') as f:
            for c in comments:
                f.write("c " + c + "\n")
            f.write("p EDGE {} {}\n".format(g.number_of_nodes(), g.number_of_edges()))
            for u, v in g.edges():
                f.write("e {} {}\n".format(u, v))
            for node in g.nodes():
                if 'c' in g.node[node]:
                    f.write("l {} {}\n".format(node, g.node[node]['c']))

    def draw_with_colors(g = nx.Graph()):
        color_map = []
        for node in g.nodes():
            if 'c' in g.node[node]:
                color_map.append(g.node[node]['c'] * 10)            
        nx.draw(g, pos = nx.spring_layout(g, scale=2), node_color=color_map, with_labels=True, cmap = plt.cm.jet)

    def write_proof(file = "proof.txt", proof = []):
        with open(file, 'w') as f:
            for p in proof:
                f.write("%s\n" % str(p))

    def zip_files(file = "archive.zip", files = []):
        with ZipFile(file, 'w') as archive:
            for f in set(files):                
                if not (os.path.exists(file) and os.path.getsize(file) > 0):        
                    raise Exception("File " + file + " not found")
                archive.write(f)
                
def find_triangle(g = nx.Graph()):
    for a in g:
        for b, c in combinations(g[a], 2):
            if b in g[c]:
                return [a, b, c]
    return []
             
    #return set(frozenset([a, b, c]) for a in g for b, c in combinations(g[a], 2) if b in g[c])


def find_isolates(g = nx.Graph()):
    isolates = []
    for n in g:
        if g.degree(n) == 0:
            isolates.append(n)
    return isolates

class ColMap:
    
    def __init__(self, g = nx.Graph(), ncolors = 40):
        
        self.ncolors = ncolors
        self.cmap = dict()
        self.cunmap = dict()
    
        i = 1
        for node in g.nodes():
            for color in range(1, ncolors + 1):            
                self.cmap[(node, color)] = i
                self.cunmap[i] = (node, color)
                i += 1    

    def enc(self, node, color):
        return self.cmap[(node, color)]

    def dec(self, node_color):
        return self.cunmap[node_color]

class ColSAT:
   
    def __init__(self, g = nx.Graph(), ncolors = 10):
        
        self.ncolors = ncolors
        self.g = g.copy()
        self.cmap = ColMap(g, ncolors)        

    def check_coloring(self):
        for n1, n2 in self.g.edges():
            if 'c' not in self.g.node[n1] or 'c' not in self.g.node[n2]:
                return False
            if self.g.node[n1]['c'] == self.g.node[n2]['c']:
                return False
        return True
    
    def apply_model(self):
        
        check = set()
        for var in self.model[self.model > 0]:        
            node, color = self.cmap.dec(var)
            self.g.node[node]['c'] = color
            if (node, color) in check:
                raise Exception("Two colors for one node???")
            else:
                check.add((node, color))
        
        self.colored = self.check_coloring()
        
        if self.colored != self.solved:
            raise Exception("Something went wrong!")
        
        return self.colored
        
    def build_cnf(self, ):
        
        self.formula = CNF()
        colors = list(range(1, self.ncolors + 1))    

        maxc = -1        # condition k<12 is not for general case
        k = 0
        for clique in nx.find_cliques(self.g):
            if len(clique)>maxc:
                maxclique = clique.copy()  
                maxc = len(clique)
            if k<12:
                k += 1
                continue
           
            col = 1
            if len(maxclique)>self.ncolors:
                print("Size of the maximal clique is {}".format(len(maxclique)))
                raise Exception("Size of the maximal clique is {} > {}".format(len(maxclique),ncolors))
            for v in maxclique:
                self.formula.append([self.cmap.enc(v, col)])
                col += 1
            break

        for n1, n2 in self.g.edges():
            for c in colors:            
                self.formula.append([-self.cmap.enc(n1, c), -self.cmap.enc(n2, c)])


        for n in self.g.nodes():
            #if not n in specials:
            self.formula.append([self.cmap.enc(n, c) for c in colors])
            #for c1 in colors:
            #    for c2 in colors:
            #        if c1 < c2:
            #            self.formula.append([-self.cmap.enc(n, c1), -self.cmap.enc(n, c2)])
        
        return self.formula
    
    def solve_cnf(self, solver = ''):
        
        triangle = find_triangle(self.g)
        assumptions = []
        if len(triangle) > 0:            
            assumptions = [self.cmap.enc(triangle[0], 1), self.cmap.enc(triangle[1], 2), self.cmap.enc(triangle[2], 3)]
            
        #Glucose3, Glucose4, Lingeling, MapleChrono, MapleCM, Maplesat, Minisat22, MinisatGH
        #with Glucose4(bootstrap_with=self.formula.clauses, with_proof=True) as ms:        
        with Lingeling(bootstrap_with=self.formula.clauses) as ms:
            self.solved = ms.solve(assumptions=assumptions)
            if self.solved:
                self.model = np.array(ms.get_model())
                self.apply_model()
            else:                
                self.proof = []#ms.get_proof()
                self.colored = False
                
        return self.solved






In [2]:
# выбор графа на вершинах гиперкуба (покрышки), удаление независимого множества,
# проверка существования раскраски

import networkx as nx

def dist(a,b):
  return sum([int(c) for c in bin(a^b).split('b')[1]])

def bin_graph(n, k, trim = []):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    for t in trim:
      if dist(t,i)>k:
        flag = False
        break
    if flag:
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def small_graph(n, k, dists = [2,4,8]):   # третий параметр - множество вершин
  g = nx.Graph()                  # по k-окрестностям которых происходит отсечение    
  for i in range(0,2**n):
    if dist(0,i) not in dists:
      continue
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)

def bin_graph1(n, k, trim = []):
  g = nx.Graph()
  for i in range(0,2**n):
    if dist(0,i)%2==1:
      continue
    flag = True
    if dist(0,i)>k:
      flag = False
      
    if flag or i in set(trim):
      g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)==k):
        g.add_edge(i,j)
  return nx.convert_node_labels_to_integers(g)


def r_graph(n, k):      # граф запретов,
  g = nx.Graph()        # ребрами соединены вершины на расстоянии >k
  for i in range(0,2**n):
    g.add_node(i)
  for i in list(g.nodes()):
    for j in g.nodes():
      if (i<j) and (dist(i,j)>k):
        g.add_edge(i,j)
  return g

def write_metis(g0,fname):    # запись графа во входном формате KaMIS
  g = nx.convert_node_labels_to_integers(g0)
  with open(fname,"w") as f:
    f.write("{} {}\n".format(g.number_of_nodes(),g.number_of_edges()))
    for v in range(g.number_of_nodes()):
      f.write(" ".join([str(i+1) for i in sorted(list(g[v]))])+"\n")    

def write_cnf(g,ncolors,filename):  # запись КНФ для SAT-решателя    
  problem = ColSAT(g, ncolors)
  problem.build_cnf().to_file(filename)

def select_mis(g,filename):  # возвращает независимое множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==0:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      


def delete_mis(g,filename):  # удаляет независимосе множество
  g1 = g.copy()
  with open(filename,"r") as f:
    l = f.readlines()
    for i in range(len(l)):
      if int(l[i])==1:
        g1.remove_node(i)    
  return nx.convert_node_labels_to_integers(g1,0)      

# размерность, расстояние, номера вершин, по окрестностям которых 
# выполняется отсечение
 
r = bin_graph(10,6,trim=[0, 984, 730, 826, 1000, 718, 857])


#g = bin_graph(9,6,trim=[0,63])  
#print(r.number_of_nodes())
#print(r.number_of_edges())
#write_metis(r,"test.graph")

#!./redumis test.graph --output test.mis --time_limit=30

#g = r.copy()
g1 = r.copy()

print(g1.number_of_nodes())
print(g1.number_of_edges())

#write_cnf(g1,10,"test.cnf")

#!./kissat1 test.cnf

nmis = 0

283
15247


In [3]:
FILENAME = "test_11"
COLORSS = 0

def is_not_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return True

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if any(["UNSATISFIABLE" in line for line in outputt]):
         return True
    return False

def is_colorable(graph, color_count, timer):
    global FILENAME, COLORSS
    COLORSS += 1
    coloring_filename = FILENAME + ".cnf"
    try:
        graph_copy = graph.copy()
        write_cnf(graph_copy, color_count, coloring_filename)
    except Exception as err:
        return False

    outputt = !timeout $timer ./kissat1 $coloring_filename
    if all(["UNSATISFIABLE" not in line for line in outputt]):
        if any(["SATISFIABLE" in line for line in outputt]):
            return True
    return False

def check_colorable_fast(graph, max_colors=100, timer=120):
    if graph.number_of_edges() == 0:
        return (None, 0)

    max_uncolorable = None
    min_colorable = None
    
    left_border = 1
    right_border = max_colors + 1
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_not_colorable(graph, color_count_to_check, timer):
            left_border = color_count_to_check
        else:
            right_border = color_count_to_check
    max_uncolorable = left_border

    left_border = 1
    right_border = max_colors
    while right_border - left_border > 1:
        color_count_to_check = (right_border + left_border) // 2
        if is_colorable(graph, color_count_to_check, timer):
            right_border = color_count_to_check
        else:
            left_border = color_count_to_check
    if right_border == max_colors:
        if is_colorable(graph, max_colors, timer):
            min_colorable = right_border 
        return (max_uncolorable, min_colorable)
    else:
        min_colorable = right_border
        return (max_uncolorable, min_colorable)
    return (None, None)

# my_graph = small_graph(10, 6)
# qq = my_graph.copy()
# check_colorable_fast(qq)

# Ниже рабочая зона (Да)#


In [81]:
FILENAME = "ISK6"

130 85 15


In [14]:
BIG_NUM = 3
COUNTER = 0
BPM = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]

def simple_cook_graph_K6(base_conf, add_conf, k_2, k_4, k_6, n=10, k=6):   # третий параметр - множество вершин
    global COUNTER, BPM
    COUNTER += 1
#     if COUNTER % 500 == 0:
#         print(BPM)
    g = nx.Graph() 
    bans = base_conf + add_conf
    pooles = k_2 + k_4 + k_6
    for i in bans:
        g.add_node(i)
    
    for i in pooles:
        flag = True
        for j in bans:
            if dist(i,j) > k or dist(i, j) == 0:
                flag = False
                break
        if not flag:
            continue
        g.add_node(i)
    for i in list(g.nodes()):
        for j in g.nodes():
            if (i<j) and (dist(i,j)==k):
                g.add_edge(i,j)
    return nx.convert_node_labels_to_integers(g)

def sort_to_check(to_check, base_conf, add_conf):
    to_to = [[a, 0] for a in to_check]
    for a in to_to:
        for b in base_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
        for b in  add_conf:
            if dist(a[0], b) == 6:
                a[1] += 1
    tbd = sorted(to_to, key=lambda x: (-x[1], x[0]))
    return [a[0] for a in tbd]


def bruteforce_nolimits_k6(to_check, base_conf, add_conf, k_2, k_4, how_much=BIG_NUM, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        # if i % 20 == 0 and how_much == BIG_NUM - 1:
        #     print(i, len(to_check))
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue

        if how_much > 1:
            bruteforce_nolimits_k6(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, 
                                     how_much=how_much-1,timer=timer, n=n, k=k)
        else:
            grph = simple_cook_graph_K6(base_conf=base_conf, add_conf=add_conf + [elem], k_2=k_2, k_4=k_4, k_6=to_check[i+1:], n=n, k=k)
            brb = is_colorable(grph, 11, timer)
            if not brb:
                print(base_conf, add_conf + [elem])
    if how_much == BIG_NUM:
        print("analysing {} finished".format(BIG_NUM))

def bruteforce_nolimits_k6_short(to_check, base_conf, add_conf, k_2, k_4, how_much=4, timer=120, n=10, k=6, fromm=0):
    global COUNTER, BPM
    to_check = sort_to_check(to_check, base_conf, add_conf)
    if how_much == BIG_NUM:
        to_check = to_check[fromm:]
    ln = len(to_check)
    for i in range(ln):
        BPM[how_much][0] = i
        BPM[how_much][1] = ln
        elem = to_check[i]
        if elem in base_conf or elem in add_conf:
            continue
        flag = True
        for el in add_conf:
            if dist(elem, el) > 6:
                flag = False
                break
        if not flag:
            continue
    
        grph = simple_cook_graph_K6(base_conf=base_conf, add_conf=add_conf + [elem], k_2=k_2, k_4=k_4, k_6=[], n=n, k=k)
        brb = is_colorable(grph, 11, timer)
        if not brb:
            if how_much == 1:
                print(base_conf, add_conf + [elem])
        if how_much > 1:
            bruteforce_nolimits_k6_short(to_check[i+1:], base_conf, add_conf + [elem], k_2=k_2,
                                     k_4=k_4, how_much=how_much-1,timer=timer, n=n, k=k)

    if how_much == BIG_NUM:
        print("analysing {}-or-less-but-at-least-1 finished".format(BIG_NUM))

In [85]:
# !!! this is late-stage thing
FILENAME = "ISK6"
kv = [0, 945, 910, 630, 219, 365, 159]
k_6 = []
k_4 = []
k_2 = []
for i in range(1024):
    if dist(i, 0) % 2 == 1 or dist(i, 0) > 6 or i in kv:
        continue
    flag = True
    for v in kv:
        if dist(v, i) > 6:
            flag = False
    if not flag:
        continue
    if dist(i, 0) == 6:
        k_6.append(i)
    elif dist(i, 0) == 4:
        k_4.append(i)
    elif dist(i, 0) == 2:
        k_2.append(i)
print(len(k_6), len(k_4), len(k_2))

120 74 13


In [ ]:
%%time
BIG_NUM=4
print("COLORED", COLORSS)
print('to check:', len(k_6), len(k_6))
bruteforce_nolimits_k6(k_6, kv, [], k_2, k_4, BIG_NUM, 20, 10, 6)
print("AFTER ALL-COLORED", COLORSS)

COLORED 1093326
to check: 120 120
[[0, 70], [8, 117], [0, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [62, 117], [0, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [18, 116], [1, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [73, 116], [1, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [32, 115], [2, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [48, 113], [4, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [12, 112], [5, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [68, 112], [5, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [23, 111], [6, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [86, 111], [6, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [36, 110], [7, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [1, 109], [8, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [54, 109], [8, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [20, 108], [9, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [86, 108], [9, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70], [43, 107], [10, 118], [0, 119], [0, 120], [0, 80]]
[[0, 70

[[0, 70], [25, 84], [32, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [11, 83], [33, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [75, 83], [33, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [58, 82], [34, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [41, 81], [35, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [25, 80], [36, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [25, 79], [37, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [18, 78], [38, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [11, 77], [39, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [11, 76], [40, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [9, 75], [41, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [11, 74], [42, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [15, 73], [43, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [22, 72], [44, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [16, 71], [45, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [24, 70], [46, 117], [1, 119], [0, 120], [0, 80]]
[[0, 70], [22, 69], [47, 117], [1, 119], 

[[0, 70], [6, 108], [6, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [62, 108], [6, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [26, 107], [7, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [93, 107], [7, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [47, 106], [8, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [15, 105], [9, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [80, 105], [9, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [35, 104], [10, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [1, 103], [11, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [60, 103], [11, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [27, 102], [12, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [99, 102], [12, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [51, 101], [13, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [29, 100], [14, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [8, 99], [15, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [72, 99], [15, 115], [3, 119], [0, 120], [0, 80]]
[[0, 70], [36, 98], [16, 115], [3, 1

[[0, 70], [31, 63], [50, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [36, 62], [51, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [48, 61], [52, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [2, 59], [54, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [26, 58], [55, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [31, 56], [57, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [2, 54], [59, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [21, 53], [60, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [36, 52], [61, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [13, 49], [64, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [34, 48], [65, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [12, 46], [67, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [40, 44], [69, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [29, 42], [71, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [16, 40], [73, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [25, 38], [75, 114], [4, 119], [0, 120], [0, 80]]
[[0, 70], [1, 35], [78, 114], [4, 119], [0

[[0, 70], [83, 88], [23, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [55, 87], [24, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [49, 86], [25, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [30, 85], [26, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [15, 84], [27, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [2, 83], [28, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [71, 83], [28, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [51, 82], [29, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [40, 81], [30, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [33, 80], [31, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [28, 79], [32, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [16, 78], [33, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [11, 77], [34, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [4, 76], [35, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [4, 75], [36, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [68, 75], [36, 112], [6, 119], [0, 120], [0, 80]]
[[0, 70], [60, 74], [37, 112], [6, 119], [0

...

[[0, 70], [33, 65], [41, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [42, 64], [42, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [45, 62], [44, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [46, 61], [45, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [46, 60], [46, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [50, 59], [47, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [0, 57], [49, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [10, 56], [50, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [21, 55], [51, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [26, 53], [53, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [31, 52], [54, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [41, 49], [57, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [2, 47], [59, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [14, 46], [60, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [23, 45], [61, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [40, 44], [62, 107], [11, 119], [0, 120], [0, 80]]
[[0, 70], [13, 42], [64, 1

[[0, 70], [25, 82], [22, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [21, 81], [23, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [10, 80], [24, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [0, 79], [25, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [72, 79], [25, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [60, 78], [26, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [55, 76], [28, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [48, 73], [31, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [56, 72], [32, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [59, 70], [34, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [3, 68], [36, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [2, 67], [37, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [6, 66], [38, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [5, 65], [39, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [11, 64], [40, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [18, 63], [41, 105], [13, 119], [0, 120], [0, 80]]
[[0, 70], [32, 62], [42, 105]

[[0, 70], [72, 83], [19, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [54, 82], [20, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [42, 81], [21, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [32, 80], [22, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [30, 78], [24, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [23, 76], [26, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [25, 73], [29, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [19, 72], [30, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [19, 71], [31, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [25, 70], [32, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [25, 69], [33, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [29, 68], [34, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [28, 67], [35, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [35, 66], [36, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [42, 65], [37, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [52, 64], [38, 103], [15, 119], [0, 120], [0, 80]]
[[0, 70], [59, 63], [39,

[[0, 70], [56, 72], [28, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [51, 71], [29, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [59, 70], [30, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [57, 69], [31, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [64, 68], [32, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [2, 66], [34, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [8, 64], [36, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [16, 63], [37, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [29, 62], [38, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [37, 61], [39, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [2, 59], [41, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [8, 57], [43, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [30, 56], [44, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [48, 55], [45, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [10, 53], [47, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [19, 52], [48, 101], [17, 119], [0, 120], [0, 80]]
[[0, 70], [30, 51], [49, 101

[[0, 70], [19, 49], [49, 99], [19, 119], [0, 120], [0, 80]]
[[0, 70], [0, 47], [51, 99], [19, 119], [0, 120], [0, 80]]
[[0, 70], [29, 46], [52, 99], [19, 119], [0, 120], [0, 80]]
[[0, 70], [3, 44], [54, 99], [19, 119], [0, 120], [0, 80]]


In [1]:
2+2


4